In [28]:
import requests
from tqdm import tqdm
import math
import itertools


def get_events(league = 'NBA', start = "2022-09-09", end = "9999-12-31"):
    '''
    Get all league events on stubhub.
    '''

    event_category = {'NBA':{'path':'nba-tickets','categoryid': "6453"},
                      'NFL':{'path':'nfl-tickets','categoryid': "5084"}}


    url = f"https://www.stubhub.com/{event_category[league]['path']}"

    querystring = {"pageIndex":"1",
                "gridFilterType":"0",
                "sortBy":"0",
                "method":"GetFilteredEvents",
                "categoryId": event_category[league]['categoryid'],
                "from": start,
                "to": end}

    response = requests.request("POST", url, params=querystring).json()

    total_pages = math.ceil((response['totalCount'] - 2*response['pageSize'])/response['pageSize'])
    items = response['items']
    item_dicts = []
    with tqdm(total=total_pages) as pbar:
        while response['remaining'] > 0:
            item_dicts.append(events)

            querystring['pageIndex'] = int(querystring['pageIndex']) + 1
            response = requests.request("POST", url, params=querystring).json()
            items = response['items']
            pbar.update(1)
            
        # append last page
        item_dicts.append(items)

    return list(itertools.chain.from_iterable(item_dicts))

In [29]:
evs = get_events('NFL')

TypeError: list indices must be integers or slices, not str

In [34]:
evs[0]

{'eventId': 150020859,
 'name': 'Washington Commanders at Detroit Lions',
 'url': '/detroit-lions-detroit-tickets-9-18-2022/event/105255466/',
 'dayOfWeek': 'Sun',
 'formattedDate': 'Sep 18',
 'formattedTime': '1:00 PM',
 'isTbd': False,
 'venueName': 'Ford Field',
 'formattedVenueLocation': 'Detroit, MI, USA',
 'countryCode': 'US',
 'formattedMinPrice': '$39',
 'userDistanceFromVenue': 711630.744915106,
 'formattedDistanceInMiles': '',
 'formattedRescheduledFromDate': '',
 'eventCountdownMessage': 'Next weekend',
 'hasActiveListings': True,
 'fastSellingEventMessage': {'message': 'This event is selling fast',
  'qualifier': 'for NFL on our site',
  'disclaimer': 'This event is in the top 10% of events in NFL when ranked based on the rate of successful sales per day over the past 7 days on our site',
  'hasValue': True,
  'featureTrackingKey': '5e1b3d03-c118-465f-b9eb-7e7af50e9289'}}

In [ ]:
event_path = '/detroit-lions-detroit-tickets-9-18-2022/event/105255466/'
url = f"https://www.stubhub.com/event/105255466"
response = requests.request("POST", url)
response = requests.request("POST", response.url)
response.text

    


In [96]:
def get_listings(eventid):
    '''
    Get all event listings on stubhub.
    '''
    url = f"https://www.stubhub.com/event/{eventid}"
    url = requests.request("POST", url,allow_redirects=True).url
    response = requests.request("POST", url).json()

    querystring = {"CurrentPage":"1"}
    total_pages = math.ceil((response['TotalCount'] - 2*response['PageSize'])/response['PageSize'])
    items = response['Items']
    item_dicts = []
    with tqdm(total=total_pages) as pbar:
        while response['ItemsRemaining'] > 0:
            item_dicts.append(items)

            querystring['CurrentPage'] = int(querystring['CurrentPage']) + 1
            response = requests.request("POST", url, params=querystring).json()
            items = response['Items']
            pbar.update(1)
            
        # append last page
        item_dicts.append(items)

    return list(itertools.chain.from_iterable(item_dicts))

In [97]:
listings = get_listings(105255466)

13it [00:06,  1.97it/s]                        


In [98]:
len(listings)

464

In [99]:
ids = [a['Id'] for a in listings]
len(ids)

464

In [101]:
len(set(ids))

413

In [72]:
import pandas as pd 

In [79]:
pd.DataFrame(listings).to_csv('test.tsv',sep='\t')